In [5]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from utils.utils import *
cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
# normalization for training from imagenet wieghts
data_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize(299),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


train_set = InceptionDataset('./data/sc09/train',data_transforms)
val_set = InceptionDataset('./data/sc09/valid',data_transforms)
print(train_set.__len__())
print(val_set.__len__())

bs=32
dataset_sizes = {'train':train_set.__len__(),'val': val_set.__len__()}
train_loader = DataLoader(dataset=train_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
val_loader = DataLoader(dataset=val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
dataloaders={'train':train_loader,'val':val_loader}



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

18620
2494


This is the transfert learning training code

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            with tqdm(dataloaders[phase], unit="batch") as tepoch: 
                batches=0
                for inputs, labels in tepoch:  
                    tepoch.set_description(f"Epoch {epoch}")
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                    batches+=1                      
                    tepoch.set_postfix(loss=running_loss/(batches*bs), accuracy=100. * running_corrects.item()/(batches*bs))
                    #hist.append([running_loss/(batches*bs),100. * running_corrects.item()/(batches*bs)])
                    
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [5]:
#init model
del_ft = models.inception_v3(pretrained=True)
model_ft.aux_logits=False
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
#train and save

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)
torch.save(model_ft.state_dict(), './INCEPTIONV3.pt')



Epoch 0/4
----------


Epoch 0: 100%|█████████████████████████████████████████| 582/582 [03:56<00:00,  2.46batch/s, accuracy=69.8, loss=0.918]


train Loss: 0.9184 Acc: 0.6983


Epoch 0: 100%|████████████████████████████████████████████| 78/78 [00:26<00:00,  2.94batch/s, accuracy=91.2, loss=0.26]


val Loss: 0.2605 Acc: 0.9126

Epoch 1/4
----------


...

val Loss: 0.1823 Acc: 0.9459

Epoch 4/4
----------


Epoch 4: 100%|████████████████████████████████████████| 582/582 [03:40<00:00,  2.64batch/s, accuracy=97.6, loss=0.0749]


train Loss: 0.0749 Acc: 0.9757


Epoch 4: 100%|███████████████████████████████████████████| 78/78 [00:24<00:00,  3.24batch/s, accuracy=94.5, loss=0.194]


val Loss: 0.1938 Acc: 0.9455

Training complete in 20m 42s
Best val Acc: 0.945870
